In [ ]:
import torch, torchaudio

#knn_vc = torch.hub.load('bshall/knn-vc', 'knn_vc', prematched=True, trust_repo=True, pretrained=True, device='cuda')
knn_vc = torch.hub.load('bshall/knn-vc', 'knn_vc', prematched=True, trust_repo=True, pretrained=True, device='cpu')

Downloading: "https://github.com/bshall/knn-vc/zipball/master" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://github.com/bshall/knn-vc/releases/download/v0.1/prematch_g_02500000.pt" to /root/.cache/torch/hub/checkpoints/prematch_g_02500000.pt
100%|██████████| 63.1M/63.1M [00:01<00:00, 33.6MB/s]


Removing weight norm...
[HiFiGAN] Generator loaded with 16,523,393 parameters.


Downloading: "https://github.com/bshall/knn-vc/releases/download/v0.1/WavLM-Large.pt" to /root/.cache/torch/hub/checkpoints/WavLM-Large.pt
100%|██████████| 1.18G/1.18G [00:29<00:00, 43.0MB/s]


WavLM-Large loaded with 315,453,120 parameters.


In [ ]:

refdir=""
import os
import librosa
import numpy as np
###turn that into a genre based ref
(sigref, rate) = librosa.load(refdir, sr=16000, mono=True,  dtype=np.float32) #sanity check
dir=""
def Choose_Genre(Genre,Gender):
  dir=dir
  if Genre=="News":
    dir=dir+"News/"
    if Gender=="Male":
      dir1=dir+"CNN MultiSpeaker/Audio/Male/"
      dir2=dir+"MSNBC MultiSpeaker/Audio/Male/"
      dirlist=[dir1,dir2]
    elif Gender=="Female":
      dir1=dir+"CNN MultiSpeaker/Audio/Female/"
      dir2=dir+"MSNBC MultiSpeaker/Audio/Female/"
      dirlist=[dir1,dir2]
  elif Genre=="TEDTalk":
    dir=dir+"TedTalks/"
    if Gender=="Male":
      dir1=dir+"Male/"
      dirlist=[dir1]
    elif Gender=="Female":
      dir1=dir+"Female/"
      dirlist=[dir1]
  elif Genre=="StandUp":
    dir=dir+"StandUp/"
    if Gender=="Male":
      dir1=dir+"Male/"
      dirlist=[dir1]
    elif Gender=="Female":
      dir1=dir+"Female/"
      dirlist=[dir1]
  elif Genre=="Documentaries":
    dir=dir+"Documentaries/"
    if Gender=="Male":
      dir1=dir+"Male/"
      dirlist=[dir1]
    elif Gender=="Female":
      dir1=dir+"Female/"
      dirlist=[dir1]
  return dirlist

DirList=Choose_Genre("StandUp","Female")
ProcessedAudio=[]
for i in DirList:
  WavFiles=os.listdir(i)
  print(WavFiles)
  for j in range(0,len(WavFiles)):
    (sig, rate) = librosa.load(i+"/"+WavFiles[j], sr=16000, mono=True,  dtype=np.float32)
    a=torch.from_numpy(sig)
    ProcessedAudio.append(torch.from_numpy(sig).unsqueeze(0))


['SU1-01.wav', 'SU1-02.wav', 'SU1-03.wav', 'SU1-04.wav', 'SU1-05.wav', 'SU1-06.wav', 'SU1-07.wav', 'SU1-08.wav', 'SU1-09.wav', 'SU1-10.wav', 'SU2-01.wav', 'SU2-02.wav', 'SU2-03.wav', 'SU2-04.wav', 'SU2-05.wav', 'SU2-07.wav', 'SU2-08.wav', 'SU2-09.wav', 'SU2-10.wav', 'SU2-11.wav', 'SU2-12.wav', 'SU3-01.wav', 'SU3-02.wav', 'SU3-03.wav', 'SU3-04.wav', 'SU3-05.wav', 'SU3-06.wav', 'SU3-07.wav', 'SU3-08.wav', 'SU3-09.wav', 'SU4-01.wav', 'SU4-02.wav', 'SU4-03.wav', 'SU4-04.wav', 'SU4-05.wav', 'SU4-06.wav', 'SU4-07.wav', 'SU4-08.wav', 'SU5-01.wav', 'SU5-02.wav', 'SU5-03.wav', 'SU5-04.wav', 'SU5-05.wav', 'SU5-06.wav', 'SU5-07.wav', 'SU5-08.wav', 'SU5-09.wav', 'SU5-10.wav', 'SU5-11.wav', 'SU5-12.wav', 'SU5-13.wav', 'SU5-14.wav', 'SU5-15.wav', 'SU6-01.wav', 'SU6-02.wav', 'SU6-03.wav', 'SU6-04.wav', 'SU6-05.wav', 'SU6-06.wav', 'SU6-07.wav', 'SU6-08.wav', 'SU6-09.wav', 'SU6-10.wav', 'SU6-11.wav', 'SU6-12.wav', 'SU6-13.wav', 'SU6-14.wav', 'SU6-15.wav', 'SU6-16.wav', 'SU7-01.wav', 'SU7-02.wav', 'SU7-

In [ ]:
# path to 16kHz, single-channel, source waveform
src_wav_path = torch.from_numpy(sigref) #god forbid that any pre-processing would be explained. Thank you code developers.
# list of paths to all reference waveforms (each must be 16kHz, single-channel) from the target speaker
ref_wav_paths = ProcessedAudio
###
matching_set=knn_vc.get_matching_set(ProcessedAudio)
torch.save(matching_set,"FemaleComedy.pt")

In [ ]:
query_seq = knn_vc.get_features(src_wav_path)
import numpy as np
import pandas as pd
for i,j in zip(ProcessedAudio,WavFiles):
  try:
    matching_set = knn_vc.get_matching_set(i)
  except:
    print(j)
    print("FAILED")
print("FINISHED")
print(type(matching_set))
vectors=[]
for i in matching_set:
  Numpee=np.array(i)
  vectors.append(Numpee)
print(len(vectors))
#df=pd.DataFrame(vectors)
filename=""
#df.to_csv(filename)

FINISHED
<class 'torch.Tensor'>
421


In [ ]:
out_wav = knn_vc.match(query_seq, matching_set, topk=50)

In [ ]:
import IPython.display as ipd

In [ ]:
ipd.Audio(sigref, rate=16000)

In [ ]:
ipd.Audio(out_wav.numpy(), rate=16000)

In [ ]:
 #print(query_seq)
#print(matching_set)
#import pandas as pd
#d={"Genre":"TEDMale","Tensor":matching_set}
#Tensors=[matching_set]
#df=pd.read_csv("/content/drive/Shareddrives/Thesis_Project_HM/Thesis_Folder_HM/Architecture/KNN Prosody/GenreProsodies.csv")
#df.loc[len(df)]=d
#print(df)
#df.to_csv(r"/content/drive/Shareddrives/Thesis_Project_HM/Thesis_Folder_HM/Architecture/KNN Prosody/GenreProsodies.csv")


In [ ]:

filename='Comedy+FemaleComedy+k=50.wav'
torchaudio.save(filename, out_wav[None], 16000)
#!pwd

<audio name="abstract-reader" controls preload src="https://github.com/bshall/knn-vc/releases/download/v0.1/david-attenborough.wav"></audio>